In [2]:
%load_ext autoreload
%autoreload 2

import os
from utils import set_seed

set_seed(1234)
os.environ['HF_HOME'] = 'hf_cache' # Don't want model files in our home directory due to disk quota

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('studio-ousia/luke-large', add_prefix_space=True)

/data2/nhanse02/thesis/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
txt = ["My name is Nick.", "Mark is a friend of Jack."]
spans = [[(11, 15)], [(0, 4), (20, 24)]]

tokenizer(txt, entity_spans=spans, return_tensors='pt', padding=True)

{'input_ids': tensor([[   0, 1308,  766,   16, 2651,  479,    2,    1,    1],
        [   0, 1190,   16,   10, 1441,    9, 2722,  479,    2]]), 'entity_ids': tensor([[2, 0],
        [2, 2]]), 'entity_position_ids': tensor([[[ 4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]],

        [[ 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
         [ 6, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'entity_attention_mask': tensor([[1, 0],
        [1, 1]])}

In [5]:
from utils import read_docred
import json

samples = read_docred(fp='/data2/nhanse02/thesis/data/train_annotated.json', rel2id=json.load(open('data/meta/rel2id.json')), tokenizer=tokenizer)

/data2/nhanse02/thesis/data/train_annotated.json: 100%|██████████| 3053/3053 [00:25<00:00, 118.68it/s]


In [7]:
from utils import collate_fn

print(collate_fn(samples[15:18])['entity_position_ids'])

tensor([[[  1,   2,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 61,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 97,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [  5,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 21,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 23,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 32,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 40,  41,  42,  43,  -1,  -1,  -1,  -1,  -1],
         [138,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [148,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 34,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 49,  50,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 52,  53,  54,  55,  -1,  -1,  -1,  -1,  -1],
         [ 58,  59,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 82,  83,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 79,  80,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ 85,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [101, 102,  -1,  -1,  -1,  -1,  -1,  -1